##### 문제 2-1 : 콤마 구분 리스트 파서 활용
* 사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력하는 프로그램을 작성하세요.

In [66]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv
from pprint import pprint

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
        template=
        """
        {subject}와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 나열해주세요.
        각 항목은 특정 장소나 활동이나 음식을 나타내는 구체적인 이름이어야 합니다.
        
        예시:
        입력: "음식"
        출력: 명동 칼국수, 부산 돼지국밥 ...

        입력: "스포츠"
        출력: 잠실 야구장, 서울 월드컵 경기장 ...

        {format_instructions}""",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)
pprint(prompt.partial_variables)

{'format_instructions': 'Your response should be a list of comma separated '
                        'values, eg: `foo, bar, baz` or `foo,bar,baz`'}


In [67]:
# 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.2
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

# "음식"에 대한 체인 호출 실행
result = chain.invoke({"subject": "음식"})

# 쉼표로 구분된 리스트 출력
print("음식 관련 목록:")
print(result)

음식 관련 목록:
['종로 비빔밥', '광화문 닭볶음탕', '부산 개미집', '전주 한옥마을 순두부찌개', '대구 서문시장 치킨']


In [ ]:
# "영화"에 대한 체인 호출 실행
result = chain.invoke({"subject": "영화"})

# 쉼표로 구분된 리스트 출력
print("영화 관련 목록:")
print(result)

영화 관련 목록:
['부산 영화의 전당', 'CGV 홍대', '롯데시네마 월드몰', '한국 영화 박물관', '충무로 영화 거리']


In [ ]:
# "스포츠"에 대한 체인 호출 실행
result = chain.invoke({"subject": "스포츠츠"})

# 쉼표로 구분된 리스트 출력
print("스포츠 관련 목록:")
print(result)

스포츠 관련 목록:
['잠실 야구장', '서울 월드컵 경기장', '부산 광안리 해수욕장(서핑)', '강원도 평창 스키장', '인천 영종도 골프장']


##### 문제 2-2 : 영화 리뷰 감정 분석기
* 영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.

In [68]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# EnumOutputParser 초기화
enumParser = EnumOutputParser(enum=Sentiment)
format_instructions = enumParser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 템플릿
template = """
당신은 영화 리뷰 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)   
print(prompt)

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립
input_variables=['text'] input_types={} partial_variables={'format_instructions': 'Select one of the following options: 긍정, 부정, 중립'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='\n당신은 영화 리뷰 텍스트 감정 분석 전문가입니다.\n다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.\n\n텍스트: "{text}"\n\n{format_instructions}\n\n중요 규칙:\n1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요\n2. 다른 설명이나 부가 설명을 추가하지 마세요\n3. 이모지나 특수문자도 포함하지 마세요\n4. 오직 하나의 단어만 출력하세요\n\n답변:'), additional_kwargs={})]


In [69]:
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=enumParser, llm=model)

print("모델 및 파서 설정 완료")

# 테스트 텍스트
texts = [
        "이 영화 정말 재미없어요. 시간 낭비였습니다.",
        "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
        "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
        "영상미가 뛰어나지만 스토리가 너무 난해했어요.",
        "뻔한 클리셰 범벅에 지루하기 짝이 없었습니다. 비추천.",
        "기대했던 것보다 훨씬 좋았어요! 반전도 신선하고 몰입감 최고.",
        "가족끼리 보기 좋은 따뜻한 영화였습니다. 잔잔한 감동이 있네요.",
        "사운드가 엉망이라 집중하기 힘들었어요. 내용은 그럭저럭이었지만."
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

모델 및 파서 설정 완료
테스트할 텍스트 8개 준비 완료


In [70]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 중립 

4. 텍스트: 영상미가 뛰어나지만 스토리가 너무 난해했어요.
   감정: 중립 

5. 텍스트: 뻔한 클리셰 범벅에 지루하기 짝이 없었습니다. 비추천.
   감정: 부정 

6. 텍스트: 기대했던 것보다 훨씬 좋았어요! 반전도 신선하고 몰입감 최고.
   감정: 긍정 

7. 텍스트: 가족끼리 보기 좋은 따뜻한 영화였습니다. 잔잔한 감동이 있네요.
   감정: 긍정 

8. 텍스트: 사운드가 엉망이라 집중하기 힘들었어요. 내용은 그럭저럭이었지만.
   감정: 부정 

=== 결과 요약 ===
성공: 8/8 (100.0%)
실패: 0/8


##### 문제 2-3: 학생 정보 구조화 시스템
* 학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.

In [102]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

# 출력 구조를 정의하는 Pydantic 모델
class StudentInformation(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInformation)

# 프롬프트 템플릿 설정
template = """
제공된 학생의 자기소개 텍스트에서 이름, 나이, 전공, 취미 리스트, 목표를 추출하여 JSON 형식으로 반환하세요.
취미는 여러 개일 수 있으며, 콤마로 구분된 리스트 형태로 추출해야 합니다.

요청: "{student_introduction_text}"

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)
print(prompt)

input_variables=['student_introduction_text'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "학생의 이름", "title": "Name", "type": "string"}, "age": {"description": "학생의 나이", "title": "Age", "type": "integer"}, "major": {"description": "학생의 전공", "title": "Major", "type": "string"}, "hobbies": {"description": "학생의 취미 리스트", "items": {"type": "string"}, "title": "Hobbies", "type": "array"}, "goal": {"description": "학생의 목표", "title": "Goal", "type": "string"}}, "required": ["name", "age", "maj

In [103]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 및 실행
student_introduction_text =  "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"student_introduction_text": student_introduction_text})

# 결과 출력
print(f"name: {output.name}")
print(f"age: {output.age}")
print(f"major: {output.major}")
print(f"hobbies: {', '.join(output.hobbies)}")
print(f"goal: {output.goal}")

name: 김민수
age: 22
major: 컴퓨터공학
hobbies: 게임하기, 영화보기, 코딩
goal: 훌륭한 개발자가 되는 것


##### 문제 2-4 : 여행 계획 분석기
* 여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.

In [109]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# 출력 구조 정의 (여행지, 기간, 예산, 추천도(1~5점), 주요 활동 리스트)
response_schemas = [
    ResponseSchema(name="destination", description="여행 후기나 계획의 여행지"),
    ResponseSchema(name="duration", description="여행의 기간"),
    ResponseSchema(name="budget", description="여행의 예산"),
    ResponseSchema(name="rating", description="5점 만점에서 예상 추천도"),
    ResponseSchema(name="activities", description="언급된 주요 활동을  JSON 배열(리스트) 형태로 출력")    
]

#파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
template = """
다음 여행 후기나 계획 텍스트를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행 후기나 계획의 여행지
	"duration": string  // 여행의 기간
	"budget": string  // 여행의 예산
	"rating": string  // 5점 만점에서 예상 추천도
	"activities": string  // 언급된 주요 활동을  JSON 배열(리스트) 형태로 출력
}
```


In [ ]:
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 테스트 리뷰 데이터
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 
정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.

"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output) #key를 알파벳 순서로 정렬하게 됨.

===== 분석 결과 =====
{'activities': ['해운대에서 바다구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '총30만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4점'}
